In [ ]:
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix

plt.rcParams['font.sans-serif'] = ['Hiragino Sans', 'Yu Gothic', 'IPAexGothic', 'Noto Sans CJK JP']
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
#データを整理
df23_24 = pd.read_csv('/Users/nakamurawataru/Documents/学校/研究室/SDSC/03.バスケ/6月送付分/【2025年度】プレイバイプレイ_23-24シーズン.csv')
df24_25 = pd.read_csv('/Users/nakamurawataru/Documents/学校/研究室/SDSC/03.バスケ/6月送付分/【2025年度】プレイバイプレイ_24-25シーズン.csv')
df_box = pd.read_csv('/Users/nakamurawataru/Documents/学校/研究室/SDSC/03.バスケ/6月送付分/【2025年度】ボックススコア.csv')
df_game = pd.read_csv('/Users/nakamurawataru/Documents/学校/研究室/SDSC/03.バスケ/6月送付分/【2025年度】試合データ.csv')
df_player = pd.read_csv('/Users/nakamurawataru/Documents/学校/研究室/SDSC/03.バスケ/6月送付分/【2025年度】選手マスタ.csv')

# プレイタイムを正規化（全角コロン、DNPなどを処理）
df_box['プレイタイム_秒'] = (
    df_box['プレイタイム']
    .astype(str)                               # 念のため文字列化
    .str.replace('：', ':', regex=False)       # 全角コロン対策
    .apply(lambda x: int(x.split(':')[0]) * 60 + int(x.split(':')[1]) if ':' in x else 0)
)

# 試合ID、チームIDごとにデータを整理
df_box_gameid = df_box.drop(columns=['ホームアウェイ','チーム名','チーム名英','選手ID','背番号','選手名','スターティングフラグ'])
df_box_gameid_sum = df_box_gameid[df_box_gameid['ピリオド区分'] == 18].groupby(['試合ID','チームID']).sum(numeric_only=True).reset_index()

# 試合IDごとに勝敗を割り当て
# 1. 試合IDごとに得点の最大値を取得
max_scores = df_box_gameid_sum.groupby('試合ID')['得点'].transform('max')

# 2. 自チームの得点が最大なら1（勝ち）、そうでなければ0（負け）
df_box_gameid_sum['勝敗'] = (df_box_gameid_sum['得点'] == max_scores).astype(int)

df_player_stats = df_box.drop(columns=['ホームアウェイ','チーム名','チーム名英','背番号','選手名','スターティングフラグ'])
df_player_stats = df_player_stats[df_player_stats['ピリオド区分'] == 18].groupby('選手ID').sum(numeric_only=True).reset_index()
df_player_stats  = df_player_stats.merge(df_player, on='選手ID', how='left')
df_player_stats['OR_per_match'] = df_player_stats['オフェンスリバウンド']/df_player_stats['プレイタイム_秒'] * 40 * 60
df_player_stats['DR_per_match'] = df_player_stats['ディフェンスリバウンド']/df_player_stats['プレイタイム_秒'] * 40 * 60

In [100]:
df_player_stats[['選手ID','プレイタイム_秒','OR_per_match','DR_per_match','オフェンスリバウンド','ディフェンスリバウンド','身長','体重']]

,選手ID,プレイタイム_秒,OR_per_match,DR_per_match,オフェンスリバウンド,ディフェンスリバウンド,身長,体重
0,8446,58749,0.653628,2.165143,16,53,174,73
1,8447,214213,0.918712,2.532059,82,226,183,83
2,8447,214213,0.918712,2.532059,82,226,183,83
3,8448,115329,1.477512,3.933096,71,189,191,90
4,8448,115329,1.477512,3.933096,71,189,191,90
...,...,...,...,...,...,...,...,...
1625,5100000087,10447,3.445965,4.824351,15,21,198,93
1626,5100000088,232857,3.700125,6.287120,359,610,206,106
1627,5100000088,232857,3.700125,6.287120,359,610,206,106
1628,5100000088,232857,3.700125,6.287120,359,610,206,106


In [103]:
df_player_stats[['選手ID','プレイタイム_秒','OR_per_match','DR_per_match','オフェンスリバウンド','ディフェンスリバウンド','身長','体重']].dtypes

選手ID              int64
プレイタイム_秒          int64
OR_per_match    float64
DR_per_match    float64
オフェンスリバウンド        int64
ディフェンスリバウンド       int64
身長                int64
体重                int64
dtype: object

In [108]:
print(df_player_stats[['身長', '体重']].std())
print(df_player_stats[['身長', '体重']].corr())

print("NaNの数:", df_player_stats['OR_per_match'].isna().sum())
print("infの数:", np.isinf(df_player_stats['OR_per_match']).sum())

身長    11.246441
体重    14.168544
dtype: float64
          身長        体重
身長  1.000000  0.896143
体重  0.896143  1.000000
NaNの数: 11
infの数: 0


In [109]:
exp_val = ['身長', '体重']
df = df_player_stats.copy()

# プレイタイムが0の行は除去してから per-match を計算（安全化）
min_playtime_sec = 300  # 例: 5分未満は除外（必要に応じて変更）
df.loc[df['プレイタイム_秒'] <= 0, 'OR_per_match'] = np.nan
df['OR_per_match'] = df['オフェンスリバウンド'] / df['プレイタイム_秒'] * 40 * 60

# フィルタ: 最低プレイタイムを満たす選手かつ説明変数が有限である行のみ採用
mask = (
    (df['プレイタイム_秒'] >= min_playtime_sec)
    & np.isfinite(df['OR_per_match'])
    & df[exp_val].notnull().all(axis=1)
)
df2 = df.loc[mask, :].copy()

print("元データ件数:", len(df), "→ 使用件数:", len(df2))
print("欠損/inf の数 (OR_per_match):", df['OR_per_match'].isna().sum(), np.isinf(df['OR_per_match']).sum())

# 説明変数・目的変数
X = df2[exp_val]
y = df2['OR_per_match']

# 標準化（DataFrame を維持）
scaler = StandardScaler()
X_scaled_arr = scaler.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled_arr, columns=exp_val, index=X.index)

# 定数項を DataFrame のまま追加
X_sm = sm.add_constant(X_scaled, has_constant='add')

# 回帰
model = sm.OLS(y, X_sm).fit()
print(model.summary())

coef_df = pd.DataFrame({
    '変数': ['定数'] + exp_val,
    '係数(標準化後)': model.params.values
})
display(coef_df)

元データ件数: 1630 → 使用件数: 1594
欠損/inf の数 (OR_per_match): 11 0
                            OLS Regression Results                            
Dep. Variable:           OR_per_match   R-squared:                       0.531
Model:                            OLS   Adj. R-squared:                  0.531
Method:                 Least Squares   F-statistic:                     901.9
Date:                Sun, 09 Nov 2025   Prob (F-statistic):          1.45e-262
Time:                        15:44:22   Log-Likelihood:                -2136.9
No. Observations:                1594   AIC:                             4280.
Df Residuals:                    1591   BIC:                             4296.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------

,変数,係数(標準化後)
0,定数,1.732864
1,身長,0.431745
2,体重,0.579325


In [ ]:
exp_val = ['身長', '体重']
df = df_player_stats.copy()

# プレイタイムが0の行は除去してから per-match を計算（安全化）
min_playtime_sec = 300  # 例: 5分未満は除外（必要に応じて変更）
df.loc[df['プレイタイム_秒'] <= 0, 'OR_per_match'] = np.nan
df['OR_per_match'] = df['オフェンスリバウンド'] / df['プレイタイム_秒'] * 40 * 60

# フィルタ: 最低プレイタイムを満たす選手かつ説明変数が有限である行のみ採用
mask = (
    (df['プレイタイム_秒'] >= min_playtime_sec)
    & np.isfinite(df['OR_per_match'])
    & df[exp_val].notnull().all(axis=1)
)
df2 = df.loc[mask, :].copy()

print("元データ件数:", len(df), "→ 使用件数:", len(df2))
print("欠損/inf の数 (OR_per_match):", df['OR_per_match'].isna().sum(), np.isinf(df['OR_per_match']).sum())

# 説明変数・目的変数
X = df2[exp_val]
y = df2['OR_per_match']

# 標準化（DataFrame を維持）
scaler = StandardScaler()
X_scaled_arr = scaler.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled_arr, columns=exp_val, index=X.index)

# 定数項を DataFrame のまま追加
X_sm = sm.add_constant(X_scaled, has_constant='add')

# 回帰
model = sm.OLS(y, X_sm).fit()
print(model.summary())

coef_df = pd.DataFrame({
    '変数': ['定数'] + exp_val,
    '係数(標準化後)': model.params.values
})
display(coef_df)